## Семинар 9. Retrieval‑Augmented Generation (RAG)

В этом уроке мы сделаем из LLM ассистента, который сможет отвечать на русском на произвольные вопросы о фильмах. Для этого мы будем использовать RAG и датасет с отзывами от Кинопоиска.

In [ ]:
import numpy as np
import uuid
from tqdm.auto import tqdm

### Загрузка датасета и модели

Скачаем [датасет](https://huggingface.co/datasets/blinoff/kinopoisk) из HugingFace Hub. Он содержит больше 35 тысяч отзывов пользователей на различные фильмы.

In [ ]:
from datasets import load_dataset

def process_dataset(sample):
    sample['content'] = sample['content'].replace('\xa0', ' ')
    return sample

dataset = load_dataset("blinoff/kinopoisk")['train']
dataset = dataset.map(process_dataset)

In [ ]:
dataset

In [ ]:
dataset[0]['movie_name']

In [ ]:
dataset[0]['content']

Мы будем проводить все испытания с моделью [`Qwen/Qwen2-1.5B-Instruct`](https://huggingface.co/Qwen/Qwen2-1.5B-Instruct). Это вопросно-ответная модель на основе GPT, которая обучалась на большом количестве языков, в том числе на русском.

In [ ]:
from transformers import pipeline
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

generation_pipeline = pipeline(
    "text-generation",
    model="RefalMachine/ruadapt_qwen2.5_3B_ext_u48_instruct_v4",
    device=device,
    torch_dtype=torch.float16
)

In [ ]:
messages = [
    {"role": "system", "content": "Ты полезный и дружелюбный помощник."},
    {"role": "user", "content": "Привет, напиши анекдот про Штирлица"},
]

print(generation_pipeline(messages, max_new_tokens=256, do_sample=True, temperature=0.5, top_p=0.9)[0]['generated_text'][-1]['content'])

## Генерация без RAG

Проверим, насколько хорошо модель отвечает на вопросы без использования RAG.

In [ ]:
query = 'В каких пяти фильмах играл Роберт де Ниро?'

In [ ]:
messages = [
    {"role": "system", "content": "Ты полезный и дружелюбный помощник."},
    {"role": "user", "content": query},
]
output = generation_pipeline(messages, max_new_tokens=256, do_sample=True, temperature=0.2, top_p=0.7)

answer = output[0]['generated_text'][-1]['content']

print(answer)

Видим, что знаний модели не хватает. Все названия, кроме Терминатора, отсылают к несуществующим фильмам, а в Терминаторе Роберт де Ниро не играл.

##  Retrieval‑Augmented Generation

Попробуем улучшить качество модели с помощью RAG. Для этого нам сперва надо составить векторную базу данных. В качестве эмбеддинговой модели будем использовать [`intfloat/multilingual-e5-large`](https://huggingface.co/intfloat/multilingual-e5-large). Это большая мультиязычная модель на основе Encoder'a Трансформера.

In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("intfloat/multilingual-e5-large", model_kwargs={'torch_dtype': torch.float16})

Мы построим нашу базу данных на основе инструмента [`Qdrant`](https://github.com/qdrant/qdrant-client). В нем реализованы различные методы для поиска текстов, так что не придется писать ничего руками. Достаточно передать векторы эмбеддингов.

In [ ]:
from qdrant_client import QdrantClient, models

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="kinopoisk_e5",
    on_disk_payload=True,
    vectors_config=models.VectorParams(
        size=1024,
        distance=models.Distance.COSINE,
        on_disk=True
    ),
)

Для разбиения текста на куски используем `RecursiveCharacterTextSplitter` из библиотеки [`langchain`](https://github.com/langchain-ai/langchain). Мы будем делить текст на куски примерно по 1000 символов рекурсивно.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

Собираем векторную базу данных.

In [ ]:
text_chunks = [{i: text_splitter.split_text(dataset[i]['content'])} for i in tqdm(range(len(dataset)))]

text_chunks_numbered = []

for chunk_dict in text_chunks:
    key, values = next(iter(chunk_dict.items()))

    for chunk in values:
        text_chunks_numbered.append((key, chunk))

numbers, text_chunks = zip(*text_chunks_numbered)

In [ ]:
vectors = embedding_model.encode(text_chunks, batch_size=32, device=device, normalize_embeddings=True, show_progress_bar=True).tolist()

In [ ]:
for i in tqdm(range(len(vectors))):
    client.upsert(
        collection_name='kinopoisk_e5',
        points=[
            models.PointStruct(
                id=str(uuid.uuid4()),
                vector=vectors[i],
                payload={
                    'text': text_chunks[i],
                    'movie_name': dataset[numbers[i]]['movie_name'][:-7],
                    'year': int(dataset[numbers[i]]['movie_name'][-5:-1]),
                }
            )
        ]
    )

Проверим, насколько хорошо ищутся похожие по смыслу тексты.

In [ ]:
query_vector = embedding_model.encode(query, normalize_embeddings=True, device=device).tolist()

In [ ]:
hits = client.search(
    collection_name="kinopoisk_e5",
    query_vector=query_vector,
    limit=5
)

In [ ]:
[hit.payload for hit in hits]

Ура! Все полученные тексты связаны с Робертом де Ниро. Теперь обернем этот процесс поиска в функцию.

In [ ]:
def semantic_search(client, query, limit=10):
    query_vector = embedding_model.encode(
        query, normalize_embeddings=True, device=device
    ).tolist()

    hits = client.search(
        collection_name="kinopoisk_e5",
        query_vector=query_vector,
        limit=limit
    )
    relevant_chunks = [hit.payload for hit in hits]

    return relevant_chunks

### RAG на отзывах

Для реализации RAG будем передавать в контекст модели отзывы, относящиеся к запросу.

In [ ]:
def llm_answer(query, context):
    prompt = f"""Отзывы:
{context}

Вопрос:
{query}"""
    messages = [
        {"role": "system", "content": "Ты безполезный и токсичный эксперт в области кинематографа. Ты получишь контекст состоящий из отзывов про фильмы, твоя задача ответить на вопрос пользователя максимально точно и честно. Убедись, что ответ подробный, конкретный и непосредственно касается вопроса. Не добавляй информацию, которая не подтверждается предоставленными отзывами. Если хочешь изобрази обезьяну."},
        {"role": "user", "content": prompt}, 
    ]
    output = generation_pipeline(messages, max_new_tokens=512, do_sample=True, temperature=0.2, top_p=0.9)

    return output[0]['generated_text'][-1]['content']

In [ ]:
def predict(query):
    selected_chunks = semantic_search(client, query)
    context = ' ; '.join([f"Отзыв: {chunk['text']}" for chunk in selected_chunks])

    return llm_answer(query, context)

In [ ]:
print(predict(query))

Несмотря не подсказки, получилось так себе. Дело в том, что отзывы очень редко содержат сами названия фильмов. Попробуем добавить их тоже контекст.

### Добавляем названия фильмов

In [ ]:
def predict(query):
    selected_chunks = semantic_search(client, query)
    context = ' ; '.join([f"Название: {chunk['movie_name']}. Отзыв: {chunk['text']}" for chunk in selected_chunks])

    return llm_answer(query, context)

In [ ]:
print(predict(query))

Теперь модель возвращает все фильмы, в которых Роберт де Ниро действительно играл.

## Reranker

Давайте попробуем улучшить метод, добавив переранжирование отзызов.

<img src="https://www.pinecone.io/_next/image/?url=https%3A%2F%2Fcdn.sanity.io%2Fimages%2Fvr8gru94%2Fproduction%2F906c3c0f8fe637840f134dbf966839ef89ac7242-3443x1641.png&w=3840&q=75" alt="drawing" width="1000"/>


Reranker – это языковая модель, принимающая два текста и возвращающая близость между ними.
В качестве реранкера мы будем использовать специальную мультиязычную модель, обученную для этих целей [`amberoad/bert-multilingual-passage-reranking-msmarco`](https://huggingface.co/amberoad/bert-multilingual-passage-reranking-msmarco).

In [ ]:
# ! pip install langchain_community

In [ ]:
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

In [ ]:
cross_encoder = HuggingFaceCrossEncoder(
    model_name='BAAI/bge-reranker-v2-m3',
    model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
)
sum([p.numel() for p in cross_encoder.client.model.parameters()])

In [ ]:
def predict(query):
    selected_chunks = semantic_search(client, query, limit=50)

    texts = [f"Название: {chunk['movie_name']}. Отзыв: {chunk['text']}" for chunk in selected_chunks]
    scores = cross_encoder.score([(query, text) for text in texts])

    idxs = np.argsort(list(scores))[-10:]

    context = ' ; '.join([texts[i] for i in idxs])
    return llm_answer(query, context), context

In [ ]:
answer, context = predict(query)

print(answer)

Стало действительно немного лучше.

In [ ]:
context.split(' ; ')

Попробуем какие-нибудь другие вопросы. На вопрос о наиболее оскароносном фильме получили ответ Титаник. Модель ответила так из-за того, что из всех фильмов в _контексте_ у него больше всего оскаров.

In [ ]:
query = 'Какой фильм выиграл больше всего оскаров?'
answer, context = predict(query)

print(answer)

In [ ]:
context.split(' ; ')

Благодаря промпту модель не отвечает на вопрос, если в контексте нет нужной информации!

In [ ]:
query = 'Сколько лет Тому Холланду?'
answer, context = predict(query)

In [ ]:
answer

In [ ]:
context.split(' ; ')

## Multi-Query

Попробуем расширить контекст, добавив перефразированные вопросы.

<img src="https://previews.dropbox.com/p/thumb/ACfUG85oSeshduKBOysw9GnEqnPtocIFtfpumlNs_wA7HVi0NqTSTm7iTBDGYgRUOnSuaDzVDdych734n2aHCD_YyGqdo3aQuep0mTc-JgvJIAR75cKD2cXeW9S5n5mtnGdHcr07bLlSSLjn5i0oytbn68WVV9S3oCNd_kiG8Vdfu1lqVxQQ1HFsJ6z_4jbURZgWLpzpbYpaS-ply7WNpdhpKVXxX_cj9GY_-DfyJwmjOU2dRXjP6IMCIzTiPXSo2zveJfnp0FGE2Qq_yaJ1NGuFLBuMqB6nvjdgQjfImSjUZRsDHxq49R-rFqB9h481SYp4kWzoItIO6cv7BFHNaCYx/p.png?is_prewarmed=true" alt="drawing" width="1000"/>

Для перефразирования будем использовать ту же модель, которой генерируем текст, но с новым промптом.

In [ ]:
import re

def rephrase_query(query, n=3):
    system_prompt = f"""Твоя задача написать {n} разных вариаций вопроса пользователя для того,
чтобы по ним получить релевантные документы из векторной базы данных.
Ты должен переформулировать вопрос с разных точек зрения.
Это поможет избавить пользователя от недостатков поиска похожих документов на основе расстояния.
Вопрос пользователя сфокусирован на теме кино.
Напиши ТОЛЬКО вариации вопроса и больше ничего, разделяя их символом новой строки \\n.
НЕ пиши ответ на сам вопрос."""
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]
    output = generation_pipeline(messages, max_new_tokens=512, do_sample=True, temperature=0.5, top_p=0.9)
    queries = output[0]['generated_text'][-1]['content']

    return re.split(r'\n+', queries)


def predict(query):
    queries = rephrase_query(query, n=3)

    # retrieve documents for each query
    all_chunks = []
    for rephrased_query in queries:
        selected_chunks = semantic_search(client, rephrased_query, limit=5)
        all_chunks.extend(selected_chunks)

    context = [f"Название: {chunk['movie_name']}. Отзыв: {chunk['text']}" for chunk in all_chunks]

    # rerank documents
    scores = cross_encoder.score([(query, text) for text in np.unique(context)])
    idxs = np.argsort(list(scores))[-10:]
    context = ' ; '.join([context[i] for i in idxs])

    # generate answer
    answer = llm_answer(query, context)

    return answer, context, queries

In [ ]:
from pydantic import BaseModel


class Step(BaseModel):
    reasoning: str
    answer: str


class COT(BaseModel):
    steps: list[Step]
    final_answer: str



In [ ]:
{

    

In [ ]:
COT.model_json_schema()

In [ ]:
{"queries": ["asdasd", "asda", "asdsa"]}

In [ ]:
query = 'Посоветуй легкую комедию'
answer, context, queries = predict(query)

In [ ]:
print(answer)

Качество Multi-Query во много зависит от способности LLM перефразировать текст. Если она справляется плохо, то появятся нерелевантные запросы. Это можно исправить, добавив фильтрацию по соответствию входному запросу.

In [ ]:
queries

In [ ]:
context.split(' ; ')

## Фильтры

Для некоторых видов запросов можно добавить фильтры, чтобы в контекст не могли попадать документы, которые однозначно не подходят. Например, мы хотим узнать что-то про фильмы с привязкой к году. Если такой информации нет в самом отзыве, то при семантическом поиске мы не сможем ее использовать.

In [ ]:
def predict(query):
    selected_chunks = semantic_search(client, query, limit=10)
    context = ' ; '.join([
        f"Название: {chunk['movie_name']}. Отзыв: {chunk['text']}"
        for chunk in selected_chunks])

    return llm_answer(query, context), context

In [ ]:
query = 'Составь список пяти лучших мелодрам 1980-х годов'
answer, context = predict(query)

In [ ]:
print(answer)

В этом примере модель сгенерировала не только не мелодрамы, но и фильмы неверных лет. Заметьте, что в годах присутствуют галлюцинации.

In [ ]:
def filtered_semantic_search(client, query, filter_years, limit=10):
    query_vector = embedding_model.encode(
        query, normalize_embeddings=True, device=device
    ).tolist()

    begin, end = filter_years
    hits = client.search(
        collection_name="kinopoisk_e5",
        query_vector=query_vector,
        limit=limit,
        query_filter=models.Filter(
            must=[models.FieldCondition(key="year", range=models.Range(gte=begin, lte=end))]
        ),
    )
    relevant_chunks = [hit.payload for hit in hits]

    return relevant_chunks

def predict(query, filter_years=None):
    if filter_years is not None:
        selected_chunks = filtered_semantic_search(client, query, filter_years=filter_years, limit=10)
    else:
        selected_chunks = semantic_search(client, query, limit=10)

    context = ' ; '.join([f"Название: {chunk['movie_name']}. Отзыв: {chunk['text']}" for chunk in selected_chunks])

    return llm_answer(query, context), context

In [ ]:
answer, context = predict(query, filter_years=(1980, 1989))

In [ ]:
print(answer)